In [ ]:
import pandas as pd
import textdistance

In [ ]:
path = r'examlpe'
dataframe_of_address = pd.read_csv(path)
addresses_list = list(dataframe_of_address[0])

In [ ]:
def find_similars_addresses_dict(single_search, address_list, threshold):

    similarity_dict = {}
    threshold = 0.9
    for source in addresses_list: 
        for target in addresses_list:
            similarity_dict[source] = set()
            similarity_score = textdistance.smith_waterman.normalized_similarity(source,target)
            if similarity_score >= threshold and source != target:
                    similarity_dict[source].add([target,similarity_score])

    for each_address in similarity_dict:
        similarity_dict[each_address].sort(key=lambda x: x[1], reverse = True)


    return similarity_dict


# Reloading and turning it into the beton format

ModuleNotFoundError: No module named 'ffcv'

In [7]:
from torchvision import models
from torchsummary import summary

vgg = models.alexnet()
summary(vgg, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25